In [17]:
import sys
print(sys.path)
import re
import os
import csv
import nltk
import sklearn

import numpy as np
import pandas as pd
import tensorflow as tf

from gensim.models import KeyedVectors

from nltk.corpus import stopwords

EMBEDDING_FILE = '/Users/swapnil/work/Kaggle/data/googleNewsEmbeddings/GoogleNews-vectors-negative300.bin'
TRAIN_FILE = '/Users/swapnil/work/Kaggle/data/quoraPairs/train.csv'
TEST_FILE = '/Users/swapnil/work/Kaggle/data/quoraPairs/test.csv'
EMBEDDING_DIM = 300


['', '/Users/swapnil/anaconda2/lib/python27.zip', '/Users/swapnil/anaconda2/lib/python2.7', '/Users/swapnil/anaconda2/lib/python2.7/plat-darwin', '/Users/swapnil/anaconda2/lib/python2.7/plat-mac', '/Users/swapnil/anaconda2/lib/python2.7/plat-mac/lib-scriptpackages', '/Users/swapnil/anaconda2/lib/python2.7/lib-tk', '/Users/swapnil/anaconda2/lib/python2.7/lib-old', '/Users/swapnil/anaconda2/lib/python2.7/lib-dynload', '/Users/swapnil/anaconda2/lib/python2.7/site-packages', '/Users/swapnil/anaconda2/lib/python2.7/site-packages/Sphinx-1.5.1-py2.7.egg', '/Users/swapnil/anaconda2/lib/python2.7/site-packages/aeosa', '/Users/swapnil/anaconda2/lib/python2.7/site-packages/IPython/extensions', '/Users/swapnil/.ipython']


In [18]:
word2vec = KeyedVectors.load_word2vec_format(EMBEDDING_FILE, \
                                            binary=True)

train = pd.read_csv(TRAIN_FILE)

In [ ]:
test = pd.read_csv(TEST_FILE)

In [19]:
train = train.sample(frac=1.0)
print('Total rows ',train.shape[0])
valid = train.iloc[range(1,train.shape[0]/8),:]
print('Valid rows ',valid.shape[0])
train = train.iloc[range(train.shape[0]/8,train.shape[0]),:]
print('Train rows ',train.shape[0])

print(train.iloc[range(1,5),:])
print('Found %s word vectors of word2vec' % len(word2vec.vocab))
print('train value counts ',train['is_duplicate'].value_counts())
print('valid value counts ',valid['is_duplicate'].value_counts())

('Total rows ', 404290)
('Valid rows ', 50535)
('Train rows ', 353754)
            id    qid1    qid2  \
87009    87009   83026   12516   
146831  146831  231854  231855   
263492  263492  380056   18860   
120830  120830  195921  195922   

                                                question1  \
87009                 Why did MS Dhoni give up captaincy?   
146831  I'm 22 and have a steady job, but I'm not happ...   
263492                  Why does the Earth have blue sky?   
120830           How do I distance myself from my family?   

                                                question2  is_duplicate  
87009     Why did MS Dhoni resign as ODI and T20 Captain?             1  
146831  Does a charged capacitor weigh more than a dis...             0  
263492                      Why is the sky blue on Earth?             1  
120830  How can I distance myself from my family and b...             1  
Found 3000000 word vectors of word2vec
('train value counts ', 0    223108
1    13

In [20]:
PRINT_STATUS_ITER = 50000
stops = set(stopwords.words("english"))
print('Test print')
buckets = [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]

def cleanText(text):
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", str(text))
    return text

class PairScore:
    def __init__(self,ind1,ind2,dist):
        self.ind1 = ind1
        self.ind2 = ind2
        self.dist = dist
    
    def __lt__(self,other):
        return self.dist > other.dist

class Pair:
    def __init__(self,vec1,vec2,ind1,ind2):
        self.vec1 = vec1
        self.vec2 = vec2
        self.ind1 = ind1
        self.ind2 = ind2

class DataInstance:
    def __init__(self,dataId,pairs,label):
        self.dataId = dataId
        self.pairs = pairs
        self.label = label
    
    
def generateClosePairs(embedding1,embedding2,smallQuestion,bigQuestion):
    listPairs = []
    # Creating all possible pairs.
    for i in range(0,embedding1.shape[0]):
        for j in range(0,embedding2.shape[0]):
            p = PairScore(i,j,sklearn.metrics.pairwise
                          .cosine_similarity(embedding1[i].reshape(1,-1),embedding2[j].reshape(1,-1))[0])
            listPairs.append(p)
    
    sortedPairs = sorted(listPairs)
    
    # Creating pairs by finding closest word to the word from shorter question.
    smallConsidered = [False for i in range(embedding1.shape[0])]
    bigConsidered = [False for i in range(embedding2.shape[0])]
    finalPairs = []
    for pair in sortedPairs:
        if smallConsidered[pair.ind1] == False:
            finalPair = Pair(embedding1[pair.ind1],embedding2[pair.ind2],pair.ind1,pair.ind2)
            finalPairs.append(finalPair)
            smallConsidered[pair.ind1] = True
            bigConsidered[pair.ind2] = True
            
    for i,considered in enumerate(bigConsidered):
        if considered == False:
            finalPair = Pair(np.zeros(EMBEDDING_DIM),embedding2[i],-1,i)
            finalPairs.append(finalPair)
            
    return finalPairs
        
    #for finalPair in finalPairs:
     #   print(smallQuestion[finalPair.ind1] if finalPair.ind1!=-1 else "-1",",",bigQuestion[finalPair.ind2])
    
def processData(data,isTrain):
    processedList = []
    maxSeqLen = 0
    print('Start of processData')
    for ind,row in train.iterrows():
        question1 = cleanText(row['question1'])
        question2 = cleanText(row['question2'])
        if isTrain == True:
            label = row['is_duplicate']
        else:
            label = None
          
        if isTrain == True:
            dataId = row['id']
        else:
            dataId = row['test_id']
    
        question1Splits = question1.lower().split()
        question2Splits = question2.lower().split()

        # Removing stopwords.
        shortQuestion1 = [w for w in question1Splits if not w in stops]
        shortQuestion2 = [w for w in question2Splits if not w in stops]

        # Deciding short question and long question.
        if len(shortQuestion1) < len(shortQuestion2):
            smallQuestion = shortQuestion1
            bigQuestion = shortQuestion2
        else:
            smallQuestion = shortQuestion2
            bigQuestion = shortQuestion1
        
        smallEmbedding = np.zeros((len(smallQuestion), EMBEDDING_DIM))
        bigEmbedding = np.zeros((len(bigQuestion), EMBEDDING_DIM))
    
        # Getting embedding vectors for each word.
        wordsBelowScore = np.zeros(len(buckets))
    
        for i in range(0,len(smallQuestion)):
            word = smallQuestion[i]
            if word in word2vec.vocab:
                smallEmbedding[i] = word2vec.word_vec(word)
    
        for i in range(0,len(bigQuestion)):
            word = bigQuestion[i]
            if word in word2vec.vocab:
                bigEmbedding[i] = word2vec.word_vec(word)
        
        pairs = generateClosePairs(smallEmbedding,bigEmbedding,smallQuestion,bigQuestion)
        processedList.append(DataInstance(dataId,pairs,label))
        if len(pairs) > maxSeqLen:
            maxSeqLen = len(pairs)
        if ind % PRINT_STATUS_ITER == 0:
            print('processData curr iter ',ind)
            
    return maxSeqLen,processedList

def createDataset(processedList,maxSeqLen):
    print('Start of createDataset')
    dataset = np.zeros((len(processedList),maxSeqLen,EMBEDDING_DIM))
    lenthSet = np.zeros(len(processedList))
    labels = np.zeros(len(processedList))
    dataIds = np.zeros(len(processedList))
    for i,processed in enumerate(processedList):
        pairs = processed.pairs
        j = 0;
        for pair in pairs:
            diffSqr = np.square(np.subtract(pair.vec1,pair.vec2))
            dataset[i,j,:] = diffSqr
            j = j + 1;
        while j < maxSeqLen:
            dataset[i,j,:] = -1 * np.ones(EMBEDDING_DIM)
            j = j + 1
        lenthSet[i] = len(pairs)
        labels[i] = processed.label
        dataIds[i] = processed.dataId
        if i % PRINT_STATUS_ITER == 0:
            print('createDataset curr iter ',i)
    return dataset,lenthSet,labels,dataIds

Test print


In [12]:
#trainMaxSeqLen,trainProcessedList = processData(train,True)    

Start of processData
('processData curr iter ', 0)
('processData curr iter ', 50000)
('processData curr iter ', 100000)
('processData curr iter ', 150000)
('processData curr iter ', 200000)
('processData curr iter ', 250000)
('processData curr iter ', 300000)
('processData curr iter ', 350000)
('processData curr iter ', 400000)


In [15]:
#np.save('/Users/swapnil/work/Kaggle/data/quoraPairs/processedData/trainProcessedList.npy',trainProcessedList)
#np.save('/Users/swapnil/work/Kaggle/data/quoraPairs/processedData/trainMaxSeqLen.npy',trainMaxSeqLen)

In [ ]:
#trainProcessedList = np.load('/Users/swapnil/work/Kaggle/data/quoraPairs/processedData/trainProcessedList.npy')
#trainMaxSeqLen = np.load('/Users/swapnil/work/Kaggle/data/quoraPairs/processedData/trainMaxSeqLen.npy')

Start of createDataset
('createDataset curr iter ', 0)
('createDataset curr iter ', 50000)
('createDataset curr iter ', 100000)


In [ ]:
#print(trainProcessedList.shape)

In [ ]:
#trainDataset,trainLengthSet,trainLabels,trainDataIds = createDataset(trainProcessedList,trainMaxSeqLen)

In [ ]:
#np.save('/Users/swapnil/work/Kaggle/data/quoraPairs/processedData/trainDataset.npy',trainDataset)
#np.save('/Users/swapnil/work/Kaggle/data/quoraPairs/processedData/trainLengthSet.npy',trainLengthSet)
#np.save('/Users/swapnil/work/Kaggle/data/quoraPairs/processedData/trainLabels.npy',trainLabels)
#np.save('/Users/swapnil/work/Kaggle/data/quoraPairs/processedData/trainDataIds.npy',trainDataIds)

In [ ]:
BATCH_SIZE = 100
graph = tf.Graph()

validDataFrame = valid.iloc[range(0,(valid.shape[0])),:]
validSeqLen,processedValidList = processData(validDataFrame,True)
validData,validSeqLen,validLabels,_ = createDataset(processedValidList,validSeqLen)

with graph.as_default():
    batchData = tf.placeholder(tf.float32, [None, maxSeqLen, EMBEDDING_DIM])
    batchSeqLength = tf.placeholder(tf.int32,[tf.shape(batchData)[0]])
    
    validData = tf.constant(validData)
    validSeqLength = tf.constant(validSeqLen)
    
    labels = tf.placeholder(tf.float32,[BATCH_SIZE])
    
    #testBatchData = tf.placeholder(tf.float32, [None, maxSeqLen, EMBEDDING_DIM])
    #testBatchSeqLength = tf.placeholder(tf.int32,[tf.shape(batchData)[0]])
    
    logisticWeights = tf.Variable(tf.truncated_normal(
      [EMBEDDING_DIM], stddev=0.1))
    
    def model(modelData,modelSeqLength):
        rnnCell = tf.contrib.rnn.BasicLSTMCell(100)
        rnnOutputs, state = tf.nn.dynamic_rnn(rnnCell, modelData, dtype=tf.float32)
        lastIndex = tf.range(tf.shape(rnnOutputs)[0])*tf.shape(rnnOutputs)[1] + (modelSeqLength - 1)
        lastOutput = tf.gather(tf.reshape(rnnOutputs, [-1, state_size]), lastIndex)
        bias = tf.placeholder(tf.float32,[1])
        logits = tf.matmul(lastOutput,logisticWeights) + bias
        return logits
    
    
    logits = model(batchData,batchSeqLength)
    loss = tf.reduce_mean(
    tf.nn.sigmoid_cross_entropy_with_logits(logits=logits, labels=labels))   
    optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)

    batchPreds = tf.nn.sigmoid(logits)
    validPreds = tf.nn.sigmoid(model(validData,validSeqLength))
    #testPreds = tf.nn.sigmoid(model(testBatchData,testBatchSeqLength))


Start of processData


In [ ]:
def accuracy(preds,labels):
    predClass = np.where(preds > 0.5,np.ones(len(preds)),np.zeros(len(preds)))
    return float(sum(predClass == labels)) / len(preds)


num_steps = 1001
batch_size = 100

saver = tf.train.Saver()
with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print('Initialized')
  
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batchDataFrame = train.iloc[range(offset,(offset + batch_size)),:]
    batchSeqLen,processedBatchList = processData(batchDataFrame,True)
    batchData,batchSeqLen,batchLabels,_ = createDataset(processedBatchList,batchSeqLen)
    
    feed_dict = {batchData : batchData, batchSeqLength : batchSeqLen, validData:validData, 
                 validSeqLength:validSeqLen, labels:batchLabels}
    _, loss, batchPreds= session.run(
      [optimizer, loss, batchPreds], feed_dict=feed_dict)
    if (step % 100 == 0):
      print('Minibatch loss at step %d: %f' % (step, loss))
      print('Minibatch accuracy: %.2f%%' % accuracy(batchPreds, batchLabels))
      print('Minibatch log loss: %.2f%%' % tf.losses.log_loss(batchLabels, batchPreds))
      print('Validation accuracy: %.2f%%' % accuracy(validPreds.eval(), validLabels))
      print('Validation log loss: %.2f%%' % tf.losses.log_loss(validPreds.eval(), validLabels))
    
    if (step % 50000 == 0):
        saver.save(sess, '/Users/swapnil/work/Kaggle/data/quoraPairs/models/firstModel/model',global_step=step)
    
   